This notebook is used to use KIM to predict heat fluxes from selected predictors using eddy covariance data.

In [1]:
# Libraries
from pathlib import Path
import pandas as pd
import numpy as np

from kim.map import KIM
from kim.data import Data
from kim.mapping_model import MLP

import jax

%load_ext autoreload
%autoreload 2


# Read the data
- The `Output_fluxes_daily.csv` file includes the observations of heat fluxes to be predicted, represented by $\mathbf{Y}$.
- The `Input_forcings_daily.csv` file includes the observations of forcing data, represented by $\mathbf{X}$.

In [2]:
# File and folder paths
f_y = Path("./data/Output_fluxes_daily.csv")
f_x = Path("./data/Input_forcings_daily.csv")


In [4]:
df_x, df_y = pd.read_csv(f_x, index_col=0),pd.read_csv(f_y, index_col=0)
y_keys, x_keys = df_y.keys(), df_x.keys()
y, x = df_y.values, df_x.values
x.shape, y.shape


((1279, 10), (1279, 3))

# Configurations

## Preliminary analysis configuration

In [5]:
# The random seed used in the statistical significance test
seed_shuffle = 1234

# The folder where the data analysis results will be saved
f_data_save = Path("./results/data_daily")


In [7]:
# Data configuration
data_params = {
    "xscaler_type": "minmax",
    "yscaler_type": "minmax",
}

# Sensitivity analysis configuration
sensitivity_params = {
    "method": "pc", "metric": "it-knn",
    "sst": True, "ntest": 100, "alpha": 0.05, "k": 3,
    "n_jobs": 20, "seed_shuffle": seed_shuffle,
    "verbose": 1
}


## Ensemble learning configuration

In [8]:
# Some basis ensemble learning configuration
Ns_train = 365
Ns_val = 365
hidden_activation = 'sigmoid'
final_activation = 'leaky_relu'
seed_ens = 1024
seed_predict = 3636
seed_dl = 10
seed_model = 100
training_verbose = 1
n_models = 100
n_jobs = 20

# Locations where the ensemble learning results will be saved
f_kim_save1 = Path("./results/map_many2many_daily")
f_kim_save2 = Path("./results/map_many2one_daily")
f_kim_save3 = Path("./results/map_many2one_cond_daily")


In [9]:
# Mapping parameters for each test below
map_configs = {
    "model_type": MLP,
    'n_model': n_models,
    'ensemble_type': 'ens_random',
    'model_hp_choices': {
        "depth": [1,3,5,6],
        "width_size": [3,6,10]
    },
    'model_hp_fixed': {
        "hidden_activation": hidden_activation,
        "final_activation": final_activation,
        "model_seed": seed_model
    },
    'optax_hp_choices': {
        'learning_rate': [0.01, 0.005, 0.003],
    },
    'optax_hp_fixed': {
        'nsteps': 300,
        'optimizer_type': 'adam',
    },
    'dl_hp_choices': {
    },
    'dl_hp_fixed': {
        'dl_seed': seed_dl,
        'num_train_sample': Ns_train,
        'num_val_sample': Ns_val,
        'batch_size': 64
    },
    'ens_seed': seed_ens,
    'training_parallel': True,
    'parallel_config': {
        'n_jobs': n_jobs, 
        'backend': 'loky',
        'verbose': 1
    },
    'device': None,
}

# Perform preliminary data analysis
The analysis include both sensitivity analysis and redundancy filtering check

In [10]:
data = Data(x, y, **data_params)
data.calculate_sensitivity(**sensitivity_params)


Using the kNN-based information theoretic metrics ...
Performing pairwise analysis to remove insensitive inputs ...


100%|██████████| 10/10 [07:35<00:00, 45.58s/it]


Performing conditional independence testing to remove redundant inputs ...


# Train the inverse mapping

Now, let's train the inverse mappings via ensemble learning. We are training three types of inverse mappings:
- `kim1`: The naive inverse mapping from all $\mathbf{Y}$ to all $\mathbf{X}$
- `kim2`: The knowledge-informed inverse mapping from sensitive $\mathbf{Y}$ to each of $\mathbf{X}$ using global sensitivity analysis
- `kim3`: The knowledge-informed inverse mapping from sensitive $\mathbf{Y}$ to each of $\mathbf{X}$ using global sensitivity analysis + redundancy filtering check


In [11]:
# Initialize three diffferent KIMs
kim1 = KIM(data, map_configs, map_option='many2many')
kim2 = KIM(data, map_configs, mask_option="sensitivity", map_option='many2one')
kim3 = KIM(data, map_configs, mask_option="cond_sensitivity", map_option='many2one')

# Train the mappings
kim1.train()
kim2.train()
kim3.train()



 Performing ensemble training in parallel with 100 model configurations...



[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
 85%|████████▌ | 256/300 [00:03<00:00, 78.29it/s]/global/cfs/cdirs/m1800/peishi/.conda/envs/kim/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 16%|█▌        | 48/300 [00:01<00:04, 51.82it/s]/global/cfs/cdirs/m1800/peishi/.conda/envs/kim/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
100%|██████████| 300/300 [00:04<00:00, 68.08it/s]


Training completes.

 Performing ensemble training in parallel with 100 model configurations...



[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:   58.8s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
/global/cfs/cdirs/m1800/peishi/.conda/envs/kim/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
100%|██████████| 300/300 [00:04<00:00, 70.04it/s]
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:   31.1s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


Training completes.

 Performing ensemble training in parallel with 100 model configurations...



...

 56%|█████▋    | 169/300 [00:02<00:01, 74.07it/s]/global/cfs/cdirs/m1800/peishi/.conda/envs/kim/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/global/cfs/cdirs/m1800/peishi/.conda/envs/kim/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
100%|██████████| 300/300 [00:05<00:00, 53.67it/s]


Training completes.

 Performing ensemble training in parallel with 100 model configurations...



[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:   51.6s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
 96%|█████████▋| 289/300 [00:04<00:00, 74.49it/s]

Training completes.


100%|██████████| 300/300 [00:04<00:00, 60.05it/s]
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:   32.3s finished


# Save the results to disk

In [12]:
# Preliminary analysis results
data.save(f_data_save)

# Inverse mapping results
kim1.save(f_kim_save1)
kim2.save(f_kim_save2)
kim3.save(f_kim_save3)
